### **Laboratorio 1**

### **Carga de Datos de Importacion Hidrocarburos**

In [20]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import shapiro

### **Datos de Importacion** 

In [ ]:
df_import_2024 = pd.read_excel("IMPORTACION-HIDROCARBUROS-VOLUMEN-2024-12.xlsx", skiprows=6)
df_import_2025 = pd.read_excel("IMPORTACION-HIDROCARBUROS-VOLUMEN-2025-05.xlsx", skiprows=6)

print(df_import_2024.columns.tolist())

['Fecha', 'Aceites lubricantes', 'Asfalto ', 'Bunker', 'Ceras', 'Combustible turbo jet', 'Diesel bajo azufre', 'Diesel ultra bajo azufre', 'Gas licuado de petróleo', 'Gasolina de aviación', 'Gasolina regular', 'Gasolina superior', 'Grasas lubricantes', 'Kerosina', 'Mezclas oleosas', 'Naftas', 'Petcoke', 'Petróleo crudo', 'Solventes', 'Butano', 'Diesel alto azufre', 'MTBE', 'Orimulsión', 'Petróleo reconstituido', 'Total importación']


In [ ]:
# Unir Alto Azufre y Bajo Azufre para Diésel
df_import_2024["Diésel"] = df_import_2024["Diesel alto azufre"] + df_import_2024["Diesel bajo azufre"]
df_import_2025["Diésel"] = df_import_2024["Diesel alto azufre"] + df_import_2024["Diesel bajo azufre"]


In [24]:
# Unir los dos datasets
df_total = pd.concat([df_import_2024, df_import_2025], ignore_index=True)

# Convertir columna de fecha (ajustar nombre si es diferente)
df_total["Fecha"] = pd.to_datetime(df_total["Fecha"], errors="coerce")

# Seleccionar columnas de interés
cols = ["Gasolina regular", "Gasolina superior", "Diésel", "Gas licuado de petróleo"]
df_comb = df_total[["Fecha"] + cols].dropna()